### ensure that a1 is being estimated correctly

"Gh is unavailable and/or charge is not 0, so estimate a2, a4 from Shock and Helgeson 1988, and a1 from Sverjensky et al 2014." Different equations from Sverjensky 2014 are used depending on whether the species is charged.

In wrm_data:
- 11.0057 (propanoic acid a1)
- 9.6992 (propanoate a1)

In DEW data:
- 14.6 (propanoic acid a1)
- 8.3786 (propanoate a1)

In [1]:
import pyCHNOSZ
import AqEquil
import AqOrg
import pandas as pd

pyCHNOSZ.thermo("WORM")

The WORM thermodynamic database has been loaded: 1639 aqueous, 1900 total species


In [2]:
# propanoic acid
Gf_propanoic = (-93450*4.184)/1000 # kJ/mol
Hf_propanoic = (-122470*4.184)/1000 # kJ/mol
Cp_propanoic = 	60.5*4.184 # J/mol K
V_propanoic = 67.9 # cm3/mol

# propanoate
Gf_propanoate = (-86780*4.184)/1000 # kJ/mol
Hf_propanoate = (-122630*4.184)/1000 # kJ/mol
Cp_propanoate = 30.9*4.184 # J/mol K
V_propanoate = 54.95 # cm3/mol

In [3]:
# calculate the entropy from the elements of crotonic acid and crotonate (in J mol-1 K-1)
Selements_propanoic = pyCHNOSZ.entropy("C3H6O2")*4.184
Selements_propanoate = pyCHNOSZ.entropy("C3H5O2-")*4.184
Selements_propanoic, Selements_propanoate

(614.4120000000005, 614.4120000000004)

In [4]:
#G = H-T(S-Selements)
#G - H = -T(S-Selements)
#(G-H)/-T = S - Selements
#(((G-H)/-T) + Selements) = S
T = 298.15
Saq_propanoic = (((Gf_propanoic*1000-Hf_propanoic*1000)/-T) + Selements_propanoic)
Saq_propanoate = (((Gf_propanoate*1000-Hf_propanoate*1000)/-T) + Selements_propanoate)
Saq_propanoic, Saq_propanoate

(207.16839778634943, 111.32160925708564)

### 3. Estimate HKF parameters

In [5]:
# estimate parameters for the revised Helgeson Kirkham Flowers (HKF) equation of
# state in order to calculate properties at higher temperatures
# and pressures.
out_dict_propanoic = AqOrg.find_HKF(
        V=V_propanoic,
        Cp=Cp_propanoic,
        Gf=Gf_propanoic,
        Hf=Hf_propanoic,
        Saq=Saq_propanoic,
        charge=0,
        J_to_cal=True,
        print_eq=True)

out_dict_propanoate = AqOrg.find_HKF(
        V=V_propanoate,
        Cp=Cp_propanoate,
        Gf=Gf_propanoate,
        Hf=Hf_propanoate,
        Saq=Saq_propanoate,
        charge=-1,
        J_to_cal=True,
        print_eq=True)

out_dict_propanoic["a1"], out_dict_propanoate["a1"]

eta = 166027.0 (angstroms*cal/mol), YBorn = -5.802e-05 (1/K), QBorn = 5.903e-07 (1/bar), XBorn = -3.09e-07 (1/K**2)

abs_protonBorn = (0.5387 * 10**5), mentioned in text after Eq 47 in Shock and Helgeson 1988

Gh is not provided and charge equals zero so estimate omega for neutral solutes from Shock and Helgeson 1990...
HKFomega = -1.7148 = (10**-5)*((-1514.4*(Saq/4.184)) + (0.34*10**5))*4.184, Eq 61 in Shock and Helgeson 1990, omega*10**-5 (J/mol)

V_solv = 1.0122 = -(HKFomega/10**-5)*QBorn*10, Eq 5 in Shock and Helgeson 1988, along with a conversion of 10 cm3 = 1 joule/bar, delta V solvation (cm3/mol)

V_nonsolv = 66.888 = V - V_solv, Eq 4 in Shock and Helgeson 1988, delta V nonsolvation (cm3/mol)

HKFsigma = 76.045 = 1.11*V_nonsolv + 1.8, Eq 87 in Shock and Helgeson, sigma (cm3/mol)

cp_solv = 15.798 = ((HKFomega/10**-5)*298.15*XBorn), Eq 35 in Shock and Helgeson 1988, dCpsolv = omega*T*X, delta Cp solvation (J/mol*K)

cp_nonsolv = 237.33 = Cp - cp_solv, Eq 29 in Shock and Helgeson 

(10.917920341249769, 9.698753322796584)

In wrm_data:
- 11.0057 (propanoic acid a1)
- 9.6992 (propanoate a1)

In DEW data:
- 14.6 (propanoic acid a1)
- 8.3786 (propanoate a1)

# untangle:
- decide whether to implement plyasunov a1 equations as "default"
- changes made to AqOrg.py are not in the git repo...
- push changes to AqOrg.py to server
- rerun Heidi's notebook

In [6]:
# Cu+
S = 9.7
V = -8
Z = 1
omega = 0.3351

# Cu+2
S = -23.2
V = -24.6
Z = 2
omega = 1.4769

# Co+3
S = -73.000
V = -43.9
Z = 3
omega = 2.7150

# MgCl+
S = -19.000
V = 1.3
Z = 1
omega = 0.8449

# CN-
S = 22.500
V = 24.2
Z = -1
omega = 1.2900

# S2-2
S = 6.800
V = 20.2
Z = -2
omega = 3.1083
	

############################
eta=1.66027E+05
Y= -5.802E-05
Q= 5.903E-07
conv=41.8393

vs_dew = -omega*((Q*10**5)*conv)

if Z > 0:
    gamma=0.94
elif Z < 0:
    gamma=0.0
if abs(Z) == 1:
    alphaz=72
elif abs(Z) == 2:
    alphaz = 141
elif abs(Z) == 3:
    alphaz = 211
elif abs(Z) == 4:
    alphaz = 286
else:
    print("error 1")
ra=(Z**2 *(eta * Y - 100.)/(S - alphaz))
ire=int(100.*ra +.5)
re=ire/100
wabsh = 0.5387E+05
wabs = (eta*Z**2)/re
wcon = wabs-Z*wabsh
vs=-wcon*Q*conv
vn=V-vs

vn

27.876826093549376

In [7]:
vs, vs_dew

(-7.676826093549376, -7.676798148095699)

In [8]:
vn_dew = V - vs_dew

vn, vn_dew

(27.876826093549376, 27.8767981480957)

In [9]:
# Sverjensky et al 2014 appendix 1, equation 8
# (also given in the official DEW excel spreadsheet)
(0.1304*abs(Z) - 0.0217)*vn + (1.4567*abs(Z)+0.6187)

10.197449118967656

Cu+
- 1.3144 (calculated from V, S, Z given in DEW data)
- 1.2958 (DEW)

Cu+2
- -1.5172 (calculated from V, S, Z given in DEW data)
- -1.4776 (DEW)

Co+3
- -8.7773 (calculated from V, S, Z given in DEW data)
- -8.7545 (DEW)

MgCl+
- 2.4435 (calculated from V, S, Z given in DEW data)
- 3.2100 (DEW)

CN-
- 5.0522 (calculated from V, S, Z given in DEW data)
- 5.0522 (DEW)

S2-2
- 10.1975 (calculated from V, S, Z given in DEW data)
- 10.1975 (DEW)